In [1]:
import sys, os

#temporary 
sys.path.insert(0, "../../../learnMSA")

sys.path.append("../../bin")
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras
from eval_model_class import PredictionGTF
from models import lstm_model, custom_cce_f1_loss, add_hmm_layer

2024-10-24 11:37:20.802210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 11:37:20.905830: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 11:37:21.397977: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/beckerf/mambaforge/envs/tiberiusdev/lib/:/home/beckerf/mambaforge/envs/tiberiusdev/lib/python3.10/site-packages/nvidia/cudnn/lib:
2024-10-24 11:37:21.398206: W tensorflow/stream_executor/platform/default/ds

In [2]:
batch_size = 4
seq_len = 9999
strand = '+'

emb=False
hmm_factor=1
trans_lstm=False

inp_data_dir = 'inp/'
out_dir = 'test_train/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

genome_path = f'{inp_data_dir}/genome.fa'
annot_path= f'{inp_data_dir}/annot.gtf'


pred_gtf = PredictionGTF( 
     seq_len=seq_len, 
    batch_size=batch_size,
    hmm=True, 
    emb=False, 
    num_hmm=1,
    hmm_factor=1,
    genome_path=genome_path,
    annot_path=annot_path, 
    softmask=True, strand=strand,
)

# load input data x_seq 
x_seq, y_seq, _ = pred_gtf.load_inp_data(    
    strand=strand, 
    chunk_coords=True, softmask=True, pad=False
)


In [3]:
# see lstm_model documentation for more arguments 
config = {
    "num_epochs": 10,
    "stride": 0,
    "units": 100,
    "filter_size": 32,
    "numb_lstm": 2,
    "numb_conv": 3,
    "dropout_rate": 0.0,
    "hmm_dense": 32,
    "pool_size": 9,
    "lr": 1e-4,
    "batch_size": batch_size,
    "w_size": seq_len,
    'output_size': 15,
    'hmm_loss_weight_mul': 0.1,
    'hmm_emit_embeddings': False,
    "hmm_dense": 32, # size of embedding for HMM input
    'hmm_share_intron_parameters': False,
    'hmm_nucleotides_at_exons': False,
    'hmm_trainable_transitions': False,
    'hmm_trainable_starting_distribution': False,
    'hmm_trainable_emissions': False, #does not affect embedding emissions, use hmm_emit_embeddings for that
    "neutral_hmm": False, # initializes an HMM without human expert bias, currently not implemented
    'hmm_factor': 99,    
    'initial_variance': 0.1,
    'temperature': 32*3,
    "loss_f1_factor": 2.0,
}

relevant_keys = ['units', 'filter_size', 'kernel_size', 
                     'numb_conv', 'numb_lstm', 'dropout_rate', 
                     'pool_size', 'stride',  
                     'output_size', 'multi_loss']

relevant_args = {key: config[key] for key in relevant_keys if key in config}
model = lstm_model(**relevant_args)
model = add_hmm_layer(model, None,
                dense_size=config['hmm_dense'], 
                pool_size=config['pool_size'],
                output_size=config['output_size'], 
                num_hmm=1,
                l2_lambda=0.,
                hmm_factor=config['hmm_factor'], 
                batch_size=config['batch_size'],
                seq_len=config['w_size'],
                initial_variance=config['initial_variance'],
                temperature=config['temperature'],
                emit_embeddings=False, 
              share_intron_parameters=config['hmm_share_intron_parameters'],
                trainable_nucleotides_at_exons=config['hmm_nucleotides_at_exons'],
                trainable_emissions=config['hmm_trainable_emissions'],
                trainable_transitions=config['hmm_trainable_transitions'],
                trainable_starting_distribution=config['hmm_trainable_starting_distribution'],
                use_border_hints=False,
                include_lstm_in_output=False,
                neutral_hmm=config['neutral_hmm'])
              
adam = Adam(learning_rate=config["lr"])
f1loss = custom_cce_f1_loss(config["loss_f1_factor"], batch_size=config["batch_size"])



model.compile(loss=f1loss, optimizer=adam, metrics=['accuracy'])
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, None, 6)]    0           []                               
                                                                                                  
 initial_conv (Conv1D)          (None, None, 32)     608         ['main_input[0][0]']             
                                                                                                  
 layer_normalization1 (LayerNor  (None, None, 32)    64          ['initial_conv[0][0]']           
 malization)                                                                                      
                                                                                                  
 conv_1 (Conv1D)                (None, None, 32)     9248        ['layer_normalization1[0][0

In [4]:
model.fit(x=x_seq, y=y_seq, 
          epochs=config["num_epochs"], 
          batch_size=config["batch_size"])

Epoch 1/10


2024-10-24 11:37:35.702880: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8800
2024-10-24 11:37:35.791551: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-24 11:37:35.792645: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-24 11:37:35.792662: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-10-24 11:37:35.793923: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-24 11:37:35.793984: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-10-24 11:37:35.817216: I tensorflow/stream_executor/cuda/c

 154/1057 [===>..........................] - ETA: 5:54 - loss: -1193.0193 - accuracy: 0.0381 - prior: -0.1017